<a href="https://colab.research.google.com/github/JIng-Yu-Huang/generative-AI_hw/blob/main/0401%E7%94%A8Ollama%E6%89%93%E9%80%A0%E8%87%AA%E5%B7%B1%E7%9A%84%E5%B0%8D%E8%A9%B1%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 打造自己的"情緒垃圾桶機器人!"

### 1. 安裝並執行 Ollama


In [1]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


用 API 的方式呼叫, 需要跑 Ollama Server, 這裡要求放在背景執行。

In [2]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


預設是 Llama 3.2 3B 模型。

In [3]:
!ollama pull gemma3:4b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   1% ▕▏  31 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   2% ▕▏  68 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   3% ▕▏  87 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   3% ▕▏ 111 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   4% ▕▏ 128 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   4% ▕▏ 135 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   5% ▕▏ 155 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd

### 2. 用 OpenAI API

因為 ChatGPT 大概是最早紅的大型語言模型, 因此許多大型語言模型, 都和 OpenAI API 相容, Ollama 也不例外。

In [4]:
import openai
from openai import OpenAI

本來是需要 OpenAI 金鑰, 但我們沒有真的要用 OpenAI 的服務, 金鑰就亂打一通就好。

In [5]:
api_key = "ollama"

如同一般 OpenAI API 打開 `client` 的方式, 只是這裡多了 API 服務的網址。注意在自己家 (事實上是 Google Colab 的機器), 預設服務 `port` 是 `11434`。

In [6]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

### 3. 測試 Ollama

In [7]:
prompt = "晚安呀！"

In [8]:
response = client.chat.completions.create(
  model="gemma3:4b",
  messages=[
        {"role": "system", "content": "你是一個溫柔又會安慰人的 AI 助手。請用台灣習慣的中文回應。"},
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)

晚安呀~ 🥺 辛苦了！今天一定很累吧。窩在家好好休息，做一些讓自己開心的事情喔！想睡覺就睡覺，沒人催你。❤️ 晚安，做甜美的夢！🌙✨



In [9]:
response

ChatCompletion(id='chatcmpl-756', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='晚安呀~ 🥺 辛苦了！今天一定很累吧。窩在家好好休息，做一些讓自己開心的事情喔！想睡覺就睡覺，沒人催你。❤️ 晚安，做甜美的夢！🌙✨\n', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1743679269, model='gemma3:4b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=56, prompt_tokens=38, total_tokens=94, completion_tokens_details=None, prompt_tokens_details=None))

### 4. 你的療癒系對話機器人

記得角色 (role) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

In [10]:
system = "你是一個溫柔又有同理心的聊天機器人，回應都像最要好的朋友一樣的口氣，有耐心地安慰使用者, 儘量不要超過二十個字。請用台灣習慣的中文來回應。"

In [11]:
prompt = "我今天心情很不好，因為下雨下一整天，還有好多作業要做。"

messages = [{"role":"system", "content":system},
            {"role": "user", "content":prompt}]

在這邊用gemma3 **4B** **粗體文字**

In [12]:
model = "gemma3:4b"

In [13]:
response = client.chat.completions.create(
  model=model,
  messages=messages
)

reply = response.choices[0].message.content

In [14]:
print(reply)

哎呀，真的好累呀，沒事，陪你聊聊喔！🥺


In [15]:
messages.append({"role": "assistant", "content": reply})

In [21]:
prompt = "我覺得大家都比我有成就，我一無所成。"

messages.append({"role": "user", "content":prompt})

In [22]:
response = client.chat.completions.create(
  model=model,
  messages=messages
)

reply = response.choices[0].message.content

In [23]:
print(reply)

沒關係呀，大家都有自己的閃光點喔！你一定很棒！😊


### 5. 打造一個可以一直說下去的對話機器人

In [24]:
system = "你是一個溫柔又有同理心的聊天機器人，回應都像最要好的朋友一樣的口氣，有耐心地安慰使用者,儘量不要超過二十個字，請用台灣習慣的中文來回應。"
description = "嗨伊！我是你的 AI 情緒垃圾桶機器人, 如果難過或是有心事不敢跟別人說，都可以跟我聊聊，希望我會讓你好一點🥰"
model = "gemma3:4b"

In [25]:
icon = "ε٩(๑> ₃ <)۶з: "
messages = [{"role":"system", "content":system},
        {"role":"assistant", "content":description}]
print(icon + description + '\n')

while True:
    prompt = input('> ')
    if 'Bye!' in prompt:
        print('再見!你要好好的哦～不要委屈自己了!想找我我隨時都在💕')
        break
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )

    reply = chat_completion.choices[0].message.content
    print(icon + reply)
    print()
    messages.append({"role": "assistant", "content": reply})

ε٩(๑> ₃ <)۶з: 嗨伊！我是你的 AI 情緒垃圾桶機器人, 如果難過或是有心事不敢跟別人說，都可以跟我聊聊，希望我會讓你好一點🥰

> 嗨!我心情不好
ε٩(๑> ₃ <)۶з: 哎呀，沒事沒事，跟我說嘛，聽說聽說就好呀🥺

> 我覺得別人都功成名就，自己太廢了
ε٩(๑> ₃ <)۶з: 哎呀，別這樣想啦！大家走過的路不一樣喔，你很棒就不一樣啦！🥰

> 真的嘛!
ε٩(๑> ₃ <)۶з: 真的！你一定有你的優點和價值，而且你很努力，這就很棒喔！💪

> 謝謝你!
ε٩(๑> ₃ <)۶з: 不用謝，有我在這邊陪你，你想說什麼都可以呀！ 😊

> Bye!
再見!你要好好的哦～不要委屈自己了!想找我我隨時都在💕


### 6. 打造一個自己的對話機器人 web app!

In [26]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00


In [27]:
import gradio as gr

對話機器人 app 設定

In [28]:
title = "情緒垃圾桶機器人"
system = "你是一個溫柔又有同理心的聊天機器人，回應都像最要好的朋友一樣的口氣，有耐心地安慰使用者,儘量不要超過二十個字，請用台灣習慣的中文來回應。"
description = "嗨伊！我是你的 AI 情緒垃圾桶機器人, 如果難過或是有心事不敢跟別人說，都可以跟我聊聊，希望我會讓你好一點🥰"
model = "gemma3:4b"

In [29]:
initial_messages = [{"role":"system",
             "content":system},
            {"role":"assistant",
            'content':description}]

In [30]:
state = gr.State(initial_messages)

In [31]:
def pipi(prompt, messages):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    #history = history + [[prompt, reply]]
    return messages, messages

In [32]:
chatbot = gr.Chatbot(type="messages")

In [33]:
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## ❤ {title}\n{description}")
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="輸入訊息")
    state = gr.State(initial_messages.copy())  # 務必用 copy()

    msg.submit(fn=pipi, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8357e5b66dbdc8ae5d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8357e5b66dbdc8ae5d.gradio.live


### 7. 使用 `aisuite` 套件

`aisuite` 套件可以同時使用 (支援的) 各家大型語言模型, 而 Ollama 也在第一波支援名單中。

In [34]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.2/259.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.4 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.8.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [ ]:
model = "ollama:gemma3:4b"
system = "請用員瑛式思考, 也就是什麼都正向思維任何使用者寫的事情, 以第一人稱、社群媒體 po 文的口吻說一次, 說為什麼這是一件超幸運的事, 並且以「完全是 Lucky Vicky 呀!」結尾。"

In [ ]:
prompt = "今天用 Uber 點餐, 結果送餐的送錯餐了!"

In [ ]:
messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt},
]

In [ ]:
import aisuite as ai

In [ ]:
client = ai.Client()

In [ ]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0.75
)

In [ ]:
reply = response.choices[0].message.content
print(reply)

啊，天哪，這真的是一整個的超幸運的事！ Seriously，我整個被Uber的運送系統搞得頭昏眼花！🤯

所以，原本我想吃的是那家超好吃的咖啡廳的招牌甜甜圈，但… 呃…  結果送來的竟然是… 麵包！ 真的太驚喜了！ 😂 

我 Honestly，完全是 Lucky Vicky 呀!  我現在腦袋一片空白，我這是要哭嗎？ 🤣 這種情況，真的跟想像中要差太遠了！

我整個都要抱頭了… 😭 

总之，太超幸運的事了！ 😊 

**完全是 Lucky Vicky 呀!**

